# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


## Calculate utility score of training set where `action` = 1 for all `resp` > 0
[training set](https://www.kaggle.com/c/jane-street-market-prediction/data?select=train.csv)

In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv', verbose=True).to_pandas()

[*] Process input parameters
  Using default 4 thread(s)
  na_strings = ["NA"]
  strip_whitespace = True
  skip_blank_lines = False
  Input is assumed to be a file name.
File "jane-street-market-prediction/train.csv" opened, size: 6192438027
[1] Prepare for reading
  ==== file sample ====
  date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,fe...
  0,0,0.00991599128470742,0.014078579320036337,0.008773048452656298,0.0013903667296804242,0.0062703...
  0,16.673514760471395,-0.0028282266279810245,-0.0032263220711053345,-0.007319485753127453,-0.01111...
  0,0,0.02513386056271574,0.02760739373246317,0.03340572152082148,0.03437973777966444,0.02397012629...
  0,0,-0.004730139135553067,-0.003272826051591346,-4.608249864155702e-4,-4.7620477938039863e-4,-0.0...
LF character (\n) found in input, \r-only newlines will be prohibited
[2] Detect parse settings
  sep=',' with 100 lines of 138 fields using quote rule 0
  Detected 138 columns
  Quote rule = 0
 

In [2]:
X['pj'] = X.weight * np.where((X.resp > 0), X.resp, 0)
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)

224162.2681796676


## Calculate utility score of the mock test set where `action` = 1 for all *predicted* `resp` > 0
[mock test set](https://www.kaggle.com/c/jane-street-market-prediction/data?select=example_test.csv)

In [3]:
y = np.where((X.resp > 0), 1, 0)
X.drop(['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id'], axis=1, inplace=True)

In [4]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
gbc = HistGradientBoostingClassifier(verbose=1).fit(X, y)

Binning 2.272 GB of training data: 13.979 s
Binning 0.252 GB of validation data: 1.010 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.61418, val loss: 0.61416, in 0.610s
[2/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.54952, val loss: 0.54949, in 0.555s
[3/100] 1 tree, 31 leaves, max depth = 12, train loss: 0.49563, val loss: 0.49559, in 0.569s
[4/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.45003, val loss: 0.44999, in 0.568s
[5/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.41110, val loss: 0.41105, in 0.540s
[6/100] 1 tree, 31 leaves, max depth = 12, train loss: 0.37754, val loss: 0.37749, in 0.512s
[7/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.34845, val loss: 0.34841, in 0.505s
[8/100] 1 tree, 31 leaves, max depth = 12, train loss: 0.32304, val loss: 0.32301, in 0.522s
[9/100] 1 tree, 31 leaves, max depth = 15, train loss: 0.30077, val loss: 0.30074, in 0.525s
[10/100] 1 tree, 31 leaves, max depth = 1

In [5]:
del X, y

In [6]:
import pandas as pd
import numpy as np
import datatable as dt
example_sample_submission = dt.fread('jane-street-market-prediction/example_sample_submission.csv').to_pandas()
example_test = dt.fread('jane-street-market-prediction/example_test.csv').to_pandas()

In [21]:
pred = gbc.predict(example_test.drop('date', 1))
pred[pred == 0]
example_sample_submission[example_sample_submission == 0]

,ts_id,action
0,0.0,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
15214,NaN,NaN
15215,NaN,NaN
15216,NaN,NaN
15217,NaN,NaN


In [ ]:
X = pd.concat([example_sample_submission.set_index('ts_id'), example_test.set_index('ts_id')], axis=1)

In [ ]:
X

In [ ]:
X['pj'] = X.weight * X.action
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)

In [ ]:
X['pj'] = X.weight * np.where((X.resp > 0), X.resp, 0)
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)